In [3]:


pip install chromadb

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp312-cp312-win_amd64.whl
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached bcrypt-4.2.1-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached durationpy-0.9-py3-none-any.whl.metadata (338 bytes)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.29.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_proto-1.29.0-py3-none-

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.38.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.3 which is incompatible.
pr-agent 0.2.4 requires fastapi==0.111.0, but you have fastapi 0.115.6 which is incompatible.
pr-agent 0.2.4 requires Jinja2==3.1.2, but you have jinja2 3.1.5 which is incompatible.
pr-agent 0.2.4 requires litellm==1.52.12, but you have litellm 1.57.0 which is incompatible.
pr-agent 0.2.4 requires openai==1.55.3, but you have openai 1.59.3 which is incompatible.
pr-agent 0.2.4 requires pydantic==2.8.2, but you have pydantic 2.10.6 which is incompatible.
pr-agent 0.2.4 requires tenacity==8.2.3, but you have tenacity 8.5.0 which is incompatible.
pr-agent 0.2.4 requires uvicorn==0.22.0, but y

In [4]:
from vanna.openai import OpenAI_Chat
from vanna.vannadb import VannaDB_VectorStore
from vanna.chromadb import ChromaDB_VectorStore

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):

    def __init__(self,config=None):
       
        ChromaDB_VectorStore.__init__(self)
        OpenAI_Chat.__init__(self,config=config)

vn = MyVanna(config={"model":"gpt-4o-mini"})


vn.connect_to_sqlite('https://vanna.ai/Chinook.sqlite')
vn.ask("What are the top 10 albums by sales?")

SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. ===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, please explain why it can't be generated. \n4. Please use the most relevant table(s). \n5. If the question has been asked and answered before, please repeat the answer exactly as it was given before. \n6. Ensure that the output SQL is SQLite-compliant and executable, and free of syntax errors. \n"}, {'

In [5]:
vn.connect_to_sqlite('chinook.sqlite')

In [7]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")
for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE [Album]
(
    [AlbumId] INTEGER  NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),
    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)
Adding ddl: CREATE TABLE [Artist]
(
    [ArtistId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])
)
Adding ddl: CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([Emp

In [8]:
training_data = vn.get_training_data()
training_data




,id,question,content,training_data_type
0,5665d55f-7b6d-5d98-8e43-6653428695fa-ddl,None,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...,ddl
1,291a67cf-a386-5e8a-b858-dee5a9060a31-ddl,None,CREATE TABLE [Artist]\n(\n [ArtistId] INTEG...,ddl
2,898268bb-3aa1-5c1d-ab43-56a338bdef6f-ddl,None,CREATE TABLE [Customer]\n(\n [CustomerId] I...,ddl
3,344578ab-d80f-52d4-976e-78a3e54d3c6d-ddl,None,CREATE TABLE [Employee]\n(\n [EmployeeId] I...,ddl
4,7bd3c98d-b531-567d-b9c1-693c7cbbd5e3-ddl,None,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...,ddl
5,044ba63a-a15d-5339-b4f1-950e93c541be-ddl,None,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...,ddl
6,5793b49d-25f0-56ac-8557-45389b6ad864-ddl,None,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...,ddl
7,3858dfec-459a-53b5-afc9-8d854748161f-ddl,None,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...,ddl
8,9444acf2-6428-5e3c-8ab1-24f92477e88e-ddl,None,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...,ddl
9,73b20f77-f874-5c2d-bf6f-94a43bdf5664-ddl,None,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...,ddl


In [10]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


In [12]:
!pip install jsbeautifier
!pip install cssbeautifier

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for jsbeautifier: filename=jsbeautifier-1.15.1-py3-none-any.whl size=94760 sha256=a9a25755d8632400d086f32979011c31ea30328704657ffb3694d74e28d6a850
  Stored in directory: c:\users\ajaykumarkv\appdata\local\pip\cache\wheels\1f\fa\5e\6ffb2063387704c5d3aa78d2a5da20326b639bcb63fe20d359
Successfully built jsbeautifier



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for cssbeautifier: filename=cssbeautifier-1.15.1-py3-none-any.whl size=30571 sha256=450047ffa6f16d6d97a671aa4c454c283a1c03965dbc93ad03376b420ce68024
  Stored in directory: c:\users\ajaykumarkv\appdata\local\pip\cache\wheels\74\c5\01\45b606fb6555931e201738da8758a8739148f301ac9166d7e3
Successfully built cssbeautifier



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
